In [1]:
#!/usr/bin/env bash

<span style="color: red; font-weight: bold">Use the following command to launch a server for this notebook:</span>

```bash
git clone git@github.com:input-output-hk/marlowe-cardano.git
cd marlowe-cardano/marlowe-runtime/
nix run ../marlowe-cli
```

Then navigate to the `examples/` folder in Jupyter and open this notebook.

# Demonstrating the Marlowe Transaction Submit Component of Marlowe Runtime

## Preliminaries

Record version numbers

In [2]:
marlowe-cli --version

marlowe-cli 0.0.9.0


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [4]:
git rev-parse HEAD

595e48a82babf64fab7ffcedb22cda4203b8e751


### Setup the faucet.

Set the location of keys.

In [5]:
TREASURY=treasury

Set the faucet.

In [6]:
FAUCET_SKEY=$TREASURY/payment.skey
FAUCET_ADDR=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDR"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select network

In a separate terminal, set up a tunnel to the Marlowe Runtime development server:
```bash
rm /tmp/preview.socket
ssh -NT \
  -L/tmp/preview.socket:/data/networks/preview/node.socket \
  -L 3717:127.0.0.1:23717 \
  -L 3718:127.0.0.1:23718 \
  -L 3719:127.0.0.1:23719 \
  -L 3721:127.0.0.1:23721 \
  -L 3723:127.0.0.1:23723 \
  54.202.238.5                                              
```

In [7]:
export CARDANO_NODE_SOCKET_PATH=/tmp/preview.socket
export CARDANO_TESTNET_MAGIC=2
MAGIC=(--testnet-magic 2)
echo "${MAGIC[@]}"

--testnet-magic 2


### Check that the reference script has been published

Check that the Marlowe semantics validator was published.

In [8]:
marlowe-cli transaction find-published


Searching for reference script at address: addr_test1vrw0tuh8l95thdqr65dmpcfqnmcw0en7v7vhgegck7gzqgswa07sw

Expected reference script hash: "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0"

Searching for reference script at address: addr_test1vpa36uuyf95kxpcleldsncedlhjru6vdmh2vnpkdrsz4u6cll9zas

Expected reference script hash: "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3"
{
    "marlowe": {
        "hash": "6a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#1"
    },
    "payout": {
        "hash": "49076eab20243dc9462511fb98a9cfb719f86e9692288139b7c91df3",
        "txIn": "087f21f109a997193421a81886ea8c6397d336d19e696457b9c5c7aefdc31873#2"
    }
}


### Participants

#### The Party

In [9]:
PARTY_SKEY="$TREASURY/john-fletcher.skey"
PARTY_VKEY="$TREASURY/john-fletcher.vkey"

Create the first party's keys, if necessary.

In [10]:
if [[ ! -e "$PARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_SKEY" --verification-key-file "$PARTY_VKEY"
fi
PARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$PARTY_VKEY")
echo "$PARTY_ADDR"

addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r


Fund the address.

In [11]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600 \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$PARTY_ADDR"

TxId "23d1e3df8af8f0a75f0b5b8ebad4e30a5b047096f6699c5625fdd1fca2a1eb5c"


#### The Counterparty

In [12]:
COUNTERPARTY_SKEY="$TREASURY/thomas-kyd.skey"
COUNTERPARTY_VKEY="$TREASURY/thomas-kyd.vkey"

Create the second party's keys, if necessary.

In [13]:
if [[ ! -e "$COUNTERPARTY_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$COUNTERPARTY_SKEY" --verification-key-file "$COUNTERPARTY_VKEY"
fi
COUNTERPARTY_ADDR=$(cardano-cli address build "${MAGIC[@]}" --payment-verification-key-file "$COUNTERPARTY_VKEY")
echo "$COUNTERPARTY_ADDR"

addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5


Fund the address.

In [14]:
marlowe-cli util fund-address \
  --out-file /dev/null \
  --submit 600  \
  --lovelace 250000000 \
  --source-wallet-credentials "$FAUCET_ADDR:$FAUCET_SKEY" \
  "$COUNTERPARTY_ADDR"

TxId "5d27aac6fefb5c3e590086f5870fd108bce6d87abee8c7acd1241e53a915a06a"


### Time computations

In [15]:
SECOND=1000
MINUTE=$((60 * SECOND))
#HOUR=$((60 * MINUTE))
#DAY=$((24 * HOUR))

In [16]:
NOW="$(($(date -u +%s) * SECOND))"
echo "$NOW"

1666209491000


## The Contract

We set the parameters for the ACTUS PAM contract.

In [17]:
MINIMUM_ADA=3000000

FIXED_POINT=1000000
PRINCIPAL=100
INTEREST_RATE=0.02
INTEREST=$(jq -n $PRINCIPAL*$INTEREST_RATE)

STATUS_DATE=$(date -d "$(date -u -R -d @$((NOW/1000)))" +"%Y-%m-%dT00:00:00")
INITIAL_EXCHANGE_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 1 year" +"%Y-01-01T00:00:00")
MATURITY_DATE=$(date -d "$(date -u -R -d @$((NOW/1000))) + 2 year" +"%Y-01-01T00:00:00")

In [18]:
yaml2json << EOI > submit.actus
scheduleConfig:
  businessDayConvention: "NULL"
  endOfMonthConvention: "EOM"
  calendar: "NC"
maturityDate: "$MATURITY_DATE"
contractId: "0"
enableSettlement: false
initialExchangeDate: "$INITIAL_EXCHANGE_DATE"
contractRole: "RPA"
penaltyType: "O"
cycleAnchorDateOfInterestPayment: "$INITIAL_EXCHANGE_DATE"
contractType: "PAM"
notionalPrincipal: $PRINCIPAL
contractPerformance: "PF"
collateralAmount: 0
dayCountConvention: "30E360"
accruedInterest: 0
statusDate: "$STATUS_DATE"
cycleOfInterestPayment: "P1YL1"
prepaymentEffect: "N"
nominalInterestRate: $INTEREST_RATE
interestCalculationBase: "NT"
EOI
cat submit.actus

{"accruedInterest":0,"collateralAmount":0,"contractId":"0","contractPerformance":"PF","contractRole":"RPA","contractType":"PAM","cycleAnchorDateOfInterestPayment":"2023-01-01T00:00:00","cycleOfInterestPayment":"P1YL1","dayCountConvention":"30E360","enableSettlement":false,"initialExchangeDate":"2023-01-01T00:00:00","interestCalculationBase":"NT","maturityDate":"2024-01-01T00:00:00","nominalInterestRate":0.02,"notionalPrincipal":100,"penaltyType":"O","prepaymentEffect":"N","scheduleConfig":{"businessDayConvention":"NULL","calendar":"NC","endOfMonthConvention":"EOM"},"statusDate":"2022-10-19T00:00:00"}


Create the contract and its initial state.

In [19]:
marlowe-cli template actus \
  --minimum-ada "$MINIMUM_ADA" \
  --party "$PARTY_ADDR" \
  --counter-party "$COUNTERPARTY_ADDR" \
  --actus-terms-file  submit.actus \
  --out-contract-file submit-1.contract \
  --out-state-file    submit-1.state

Since we are testing, we don't really want to wait months or years for timeouts, so we edit the contract file to change the maturity date to 15 minutes from now and the initial deposit to 10 minutes from now.

In [20]:
sed -i \
  -e "s/$(jq '.timeout' submit-1.contract)/$((NOW + 10 * MINUTE))/" \
  -e "s/$(($(date -d "$MATURITY_DATE" -u +%s) * SECOND))/$((NOW + 15 * MINUTE))/" \
  submit-1.contract

View the contract.

In [21]:
json2yaml submit-1.contract

timeout: 1666210091000
timeout_continuation: close
when:
- case:
    deposits:
      negate:
        negate: 100000000
    into_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
  then:
    from_account:
      address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    pay: 100000000
    then:
      timeout: 1666210391000
      timeout_continuation: close
      when:
      - case:
          deposits: 2000000
          into_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
        then:
          from_account:
            address: addr_test1vr7n0zzth5zycuh972w7rdm

View the initial state.

In [22]:
json2yaml submit-1.state

accounts:
- - - address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
    - currency_symbol: ''
      token_name: ''
  - 3000000
boundValues: []
choices: []
minTime: 1


## Run the Contract

### Transaction 1. Create the contract

First we create a `.marlowe` file that contains the initial information needed to run the contract.

In [23]:
marlowe-cli run initialize \
  --contract-file submit-1.contract \
  --state-file    submit-1.state    \
  --out-file      submit-1.marlowe  \
  --print-stats


Validator size: 12505
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18515100, exBudgetMemory = ExMemory 80600}


Build the transaction.

In [24]:
TX_1=$(
marlowe-cli run auto-execute \
  --marlowe-out-file submit-1.marlowe \
  --required-signer "$PARTY_SKEY" \
  --change-address "$PARTY_ADDR" \
  --out-file submit-1.txbody \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_1 = $TX_1"


Fee: Lovelace 220457
Size: 1133 / 16384 = 6%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%
TX_1 = 6938fb89728e2ccc026b38e8c196cdfeed8b8b175873c44a5592f037455b9c14


Sign the transaction.

In [25]:
cardano-cli transaction sign \
  --tx-body-file submit-1.txbody \
  --out-file     submit-1.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [26]:
marlowe submit submit-1.tx

"{\"blockHeaderHash\":\"23b3337580a3bc1f41a257223aa9a70e161c87c0420fe92cc55eea4063ee982e\",\"blockNo\":286728,\"slotNo\":6206314}"


The contract is identified by its first UTxO.

In [27]:
CONTRACT_ID="$TX_1#1"
echo "$CONTRACT_ID"

6938fb89728e2ccc026b38e8c196cdfeed8b8b175873c44a5592f037455b9c14#1


### Transaction 2. Party deposits loan amount

In [28]:
marlowe-cli run prepare \
  --marlowe-file submit-1.marlowe \
  --out-file     submit-2.marlowe \
  --deposit-account "$PARTY_ADDR" \
  --deposit-party "$PARTY_ADDR" \
  --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999}), txInputs = [NormalInput (IDeposit "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\"" (Token "" "") 100000000)]}

Datum size: 605
Payment 1
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Ada: Lovelace {getLovelace = 100000000}


Build the transaction.

In [29]:
TX_2=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_1#1" \
  --marlowe-in-file  submit-1.marlowe \
  --marlowe-out-file submit-2.marlowe \
  --required-signer "$PARTY_SKEY" \
  --change-address "$PARTY_ADDR" \
  --out-file submit-2.txbody \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_2 = $TX_2"


Fee: Lovelace 1294779
Size: 14464 / 16384 = 88%
Execution units:
  Memory: 6287250 / 14000000 = 44%
  Steps: 1732252213 / 10000000000 = 17%
TX_2 = 6c2bb9fde3ca5a39c5115c69fe18516161ba456ccd6e63d4907451bc60ad7dd4


Sign the transaction.

In [30]:
cardano-cli transaction sign \
  --tx-body-file submit-2.txbody \
  --out-file     submit-2.tx \
  --signing-key-file "$PARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [31]:
marlowe submit submit-2.tx

"{\"blockHeaderHash\":\"bf9365fa36230e56fbb4db80f7825e9d652f708c5a2f4d11ed9ae1cfdeb8c659\",\"blockNo\":286729,\"slotNo\":6206335}"


### Transaction 3. Counterparty repays the loan's interest

In [32]:
marlowe-cli run prepare \
  --marlowe-file submit-2.marlowe \
  --out-file     submit-3.marlowe \
  --deposit-account "$COUNTERPARTY_ADDR" \
  --deposit-party "$COUNTERPARTY_ADDR" \
  --deposit-amount "$((INTEREST*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 2000000)]}

Datum size: 349
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 2000000}


Build the transaction.

In [33]:
TX_3=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_2#1" \
  --marlowe-in-file  submit-2.marlowe \
  --marlowe-out-file submit-3.marlowe \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$COUNTERPARTY_ADDR" \
  --out-file submit-3.txbody \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_3 = $TX_3"


Fee: Lovelace 1233027
Size: 13952 / 16384 = 85%
Execution units:
  Memory: 5815658 / 14000000 = 41%
  Steps: 1565634381 / 10000000000 = 15%
TX_3 = 898991a3384f5bdd9bd7886c706c42787c7f4821661f672aae24cab282b554d3


Sign the transaction.

In [34]:
cardano-cli transaction sign \
  --tx-body-file submit-3.txbody \
  --out-file     submit-3.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [35]:
marlowe submit submit-3.tx

"{\"blockHeaderHash\":\"f9fa0a536ca27a6231fbbadb90d263a5d4e08798990b1c465740d5f787c0fa1a\",\"blockNo\":286730,\"slotNo\":6206348}"


### Transaction 4. Counterparty repays the loan's principal

In [36]:
marlowe-cli run prepare \
  --marlowe-file submit-3.marlowe \
  --out-file     submit-4.marlowe \
  --deposit-account "$COUNTERPARTY_ADDR" \
  --deposit-party "$COUNTERPARTY_ADDR" \
  --deposit-amount "$((PRINCIPAL*FIXED_POINT))" \
  --invalid-before "$((NOW-5*MINUTE))" \
  --invalid-hereafter "$((NOW+9*MINUTE))" \
  --print-stats

Rounding  `TransactionInput` txInterval boundries to:(POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999})
TransactionInput {txInterval = (POSIXTime {getPOSIXTime = 1666209191000},POSIXTime {getPOSIXTime = 1666210031999}), txInputs = [NormalInput (IDeposit "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\"" (Token "" "") 100000000)]}

Datum size: 30
Payment 1
  Acccount: "\"addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 100000000}
Payment 2
  Acccount: "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Payee: Party "\"addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r\""
  Ada: Lovelace {getLovelace = 3000000}


Build the transaction.

In [37]:
TX_4=$(
marlowe-cli run auto-execute \
  --tx-in-marlowe "$TX_3#1" \
  --marlowe-in-file  submit-3.marlowe \
  --marlowe-out-file submit-4.marlowe \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$COUNTERPARTY_ADDR" \
  --out-file submit-4.txbody \
  --print-stats \
| sed -e 's/^TxId "\(.*\)"$/\1/'
)
echo "TX_4 = $TX_4"


Fee: Lovelace 1078968
Size: 13273 / 16384 = 81%
Execution units:
  Memory: 4210566 / 14000000 = 30%
  Steps: 1127777514 / 10000000000 = 11%
TX_4 = 80120eac4d6942ff337d91aadbca3f33503a5872d4f69fc22e76490467bafdf2


Sign the transaction.

In [38]:
cardano-cli transaction sign \
  --tx-body-file submit-4.txbody \
  --out-file     submit-4.tx \
  --signing-key-file "$COUNTERPARTY_SKEY"

Submit the transaction using Marlowe Runtime.

In [39]:
marlowe submit submit-4.tx

"{\"blockHeaderHash\":\"89a29680ab5f259d45aeb822554c2ca1962b09e946b47fd182e17721b3cc3a83\",\"blockNo\":286731,\"slotNo\":6206358}"


The contract has closed.

In [40]:
jq '.tx.contract' submit-4.marlowe | json2yaml

close
...


## Cleanup

Consolidate the UTxOs at the addresses.

In [41]:
marlowe-cli util clean \
  --change-address "$PARTY_ADDR" \
  --required-signer "$PARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "1c9668e21c149fa97cfadf02f9c882df59a1bba8a406f5844ecd75fee910e578"


In [42]:
marlowe-cli util clean \
  --change-address "$COUNTERPARTY_ADDR" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "d308a2e03ec31a7bda7db01a5f3704f43a74539eae92a75def8932d1366ff798"


Send the funds back to the faucet.

In [43]:
marlowe-cli transaction simple \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$PARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --tx-in "$(marlowe-cli util select --lovelace-only 1 "$COUNTERPARTY_ADDR" | sed -n -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;1p')" \
  --required-signer "$PARTY_SKEY" \
  --required-signer "$COUNTERPARTY_SKEY" \
  --change-address "$FAUCET_ADDR" \
  --out-file /dev/null \
  --submit 600

TxId "c0cb5cb98f7beb5e93e4778a4cb30558707d15f30d138302cff1c6914981c621"


See that the funds have been returned to the faucet.

In [44]:
cardano-cli query utxo "${MAGIC[@]}" --address "$PARTY_ADDR" --address "$COUNTERPARTY_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
